In [ ]:
import numpy as np
from scipy.stats import f

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import f
from math import sqrt
import statistics
import pandas as pd
from scipy.stats import shapiro, levene, bartlett
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importamos los datos

In [ ]:
# Cargar los datos desde el archivo CSV
datos = pd.read_csv("/content/drive/MyDrive/Proyecto de Inferencia /Reto2.csv")

# Dividir los datos en grupos
grupo_a = datos[datos['Tratamiento'] == 'A']['Eficacia']
grupo_b = datos[datos['Tratamiento'] == 'B']['Eficacia']
grupo_c = datos[datos['Tratamiento'] == 'C']['Eficacia']

#Primer Supuesto : Las Poblaciones se Distribuyen Normal


Para comprobar que nuestros datos provienen de una distribución normal usaremos la prueba de Shapiro Wilk

Hipótesis nula (H0): La muestra proviene de una población con una distribución normal.

Hipótesis alternativa (H1): La muestra no proviene de una población con una distribución normal.

In [ ]:
# Verificar normalidad en cada grupo
shapiro_test_grupo_a = shapiro(grupo_a)
shapiro_test_grupo_b = shapiro(grupo_b)
shapiro_test_grupo_c = shapiro(grupo_c)

print("Prueba de Shapiro-Wilk para Grupo A:", shapiro_test_grupo_a)
print("Prueba de Shapiro-Wilk para Grupo B:", shapiro_test_grupo_b)
print("Prueba de Shapiro-Wilk para Grupo C:", shapiro_test_grupo_c)

Prueba de Shapiro-Wilk para Grupo A: ShapiroResult(statistic=0.9823750257492065, pvalue=0.6555035710334778)
Prueba de Shapiro-Wilk para Grupo B: ShapiroResult(statistic=0.9870157837867737, pvalue=0.8535816669464111)
Prueba de Shapiro-Wilk para Grupo C: ShapiroResult(statistic=0.9626771211624146, pvalue=0.11509501188993454)


Usando una significancia del 0,05
podemos ver que para las tres muestras los p-valores son mayores que la significancia. Por lo tanto no hay suficiente evidencia cientifica para rechazar la hipótesis nula, asi que ambas muestras provienen de una distribución normal.

# Segundo Supuesto: Homogeneidad de Varianzas

Para verificar el supuesto que las varianzas de los datos son iguales usaremos el test Bartlett.

Hipótesis nula (H0): Las varianzas de todos los grupos son iguales (homogeneidad de varianzas).

Hipótesis alternativa (H1): Al menos una de las varianzas es diferente.

In [ ]:
# Verificar homogeneidad de varianzas entre los grupos
levene_test = levene(grupo_a, grupo_b, grupo_c)
print("Prueba de Levene para homogeneidad de varianzas:", levene_test)

print("se cumplen los supuestos ")

Prueba de Levene para homogeneidad de varianzas: LeveneResult(statistic=1.1387721652602372, pvalue=0.3230201877430276)
se cumplen los supuestos 


In [ ]:
# Realizar la prueba de Bartlett
estadistica_bartlett, p_valor_bartlett = bartlett(grupo_a,grupo_b,grupo_c)

# Imprimir los resultados
print("Estadística de prueba de Bartlett:", estadistica_bartlett)
print("Valor p de la prueba de Bartlett:", p_valor_bartlett)

# Interpretar los resultados
nivel_significancia = 0.05
if p_valor_bartlett < nivel_significancia:
    print("Las varianzas de los grupos son diferentes (rechazamos H0)")
else:
    print("Las varianzas de los grupos son iguales (no rechazamos H0)")

Estadística de prueba de Bartlett: 3.5258958194536407
Valor p de la prueba de Bartlett: 0.17153843791121673
Las varianzas de los grupos son iguales (no rechazamos H0)


Usando una significancia del 0,05 podemos ver que el p-valor es mayor que la significancia. Por lo tanto no hay suficiente evidencia cientifica para rechazar la hipótesis nula, las varianzas de las muestras son iguales.

**Ya que los supuestos se cumplieron, podemos realizar la prueba ANOVA con normalidad sin necesidad de transformar los datos**

#Prueba ANOVA

In [ ]:
#:p
import numpy as np
from scipy.stats import f_oneway

# Realizar la prueba ANOVA
f_statistic, p_value = f_oneway(grupo_a, grupo_b, grupo_c)

# Imprimir los resultados
print("Estadístico F:", f_statistic)
print("Valor p:", p_value)

# Interpretación del resultado
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula. Hay diferencias significativas entre los grupos.")
else:
    print("No se puede rechazar la hipótesis nula. No hay diferencias significativas entre los grupos.")


Estadístico F: 12.02635807540055
Valor p: 1.4546916965660867e-05
Se rechaza la hipótesis nula. Hay diferencias significativas entre los grupos.


In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd

def anova(a, b, c, sign=0.05):
    # Tamaño de cada grupo
    tamaños_grupos = [len(a), len(b), len(c)]

    # Media de cada grupo
    media_a = np.mean(a)
    media_b = np.mean(b)
    media_c = np.mean(c)

    # Media de todos los datos
    todos_los_datos = np.concatenate([a, b, c])
    media_total = np.mean(todos_los_datos)

    # Suma de cuadrados total (STC)
    stc = np.sum((todos_los_datos - media_total)**2)

    # Suma de cuadrados entre grupos (SCTR)
    sctr = np.sum(tamaños_grupos * (np.array([media_a, media_b, media_c]) - media_total)**2)

    # Suma de cuadrados dentro de los grupos (SCD)
    scd = np.sum((a - media_a)**2) + np.sum((b - media_b)**2) + np.sum((c - media_c)**2)

    # Suma de cuadrados de los errores (SCE)
    sce = stc - sctr

    # Grados de libertad
    gl_num = len(tamaños_grupos) - 1
    gl_dem = len(todos_los_datos) - len(tamaños_grupos)

    # Calcular la estadística F
    F = ((sctr / gl_num) / (sce / gl_dem))

    # Calcular el valor p
    valor_p = 1 - stats.f.cdf(F, gl_num, gl_dem)  # Corrección aquí

    # Imprimir resultados
    if sign > valor_p:
        print("Hay suficiente evidencia para rechazar Ho: Las medias poblacionales son diferentes")
    else:
        print("No hay suficiente evidencia para rechazar Ho: Las medias poblacionales son iguales")

    # Tabla ANOVA
    datos = {
        'Fuente Var': ["Tratamientos", "Error", "Total"],
        'Suma Cuad': [sctr, sce, stc],
        'Gl': [gl_num, gl_dem, gl_num + gl_dem],
        'Cuadr Med': [(sctr / gl_num), (sce / gl_dem), ''],
        'F': ['', F, ''],
        'Valor-p': ['', valor_p, '']
    }
    # Crear DataFrame con los datos de la tabla ANOVA
    tabla_anova = pd.DataFrame(datos)
    print("\nTabla ANOVA:")
    print(tabla_anova)

# Asegúrate de reemplazar grupo_a, grupo_b, grupo_c con tus datos reales
anova(grupo_a, grupo_b, grupo_c)


Hay suficiente evidencia para rechazar Ho: Las medias poblacionales son diferentes

Tabla ANOVA:
     Fuente Var     Suma Cuad   Gl    Cuadr Med          F   Valor-p
0  Tratamientos   3429.288719    2  1714.644359                     
1         Error  20958.358238  147   142.573866  12.026358  0.000015
2         Total  24387.646957  149                                  
